In [9]:
import asyncio
import json
import requests


from app.utility.helps import Bob

In [10]:
bob = Bob()
settings = bob.get_settings()
headers = bob.get_context()

In [ ]:
api_url = 'https://api.powerbi.com/v1.0/myorg/gateways'
response = requests.get(url=api_url, headers=headers)
results = response.json()
print(response.status_code)
gateways = list()
for gateway in results['value']:
    gateways.append(gateway.get("id"))


In [ ]:
r = list()
for id in gateways:
    response = requests.get(f'https://api.powerbi.com/v1.0/myorg/gateways/{id}/datasources', headers=headers)
    doc_results = response.json()
    r.append(doc_results['value'])   


In [ ]:
gateways = list()

for i in range(0, len(r)):
    if isinstance(r[i], list):
        for j in range(0, len(r[i])):
            gateways.append(r[i][j])
    else:
        gateways.append(r[i])

In [ ]:
gateways

In [ ]:
users = list()

for i in range(0,len(gateways)):
    api_users = f'https://api.powerbi.com/v1.0/myorg/gateways/{gateways[i]["gatewayId"]}/datasources/{gateways[i]["id"]}/users'
    response = requests.get(api_users, headers=headers)
    results = response.json()
    results['value'][0]['datasourceId'] = gateways[i]["id"]
    results['value'][0]['gatewayId'] = gateways[i]["gatewayId"]
    users.append(results['value'][0])
    

## Datasource Connectivity  

This only reports back connectivity to the datasource as an error. Also, you need to use the response.text as the JSON is not populated. A successful connection will report back as HTTP 200 while a failed connection will result in HTTP 400

In [ ]:
## This will be used for a template dictionary for a successful connection to the datasource
status_base = {   
    "datasourceId":"",
    "gatewayId":"",
    "error":
    {
        "code":"success",
        "pbi.error":{ 
            "code":"0",
            "parameters":{},
            "details":[],
            "exceptionCulprit":0
        }
    }
}


In [ ]:
status  = list()
for i in range(0,len(gateways)):
    api_status = f'https://api.powerbi.com/v1.0/myorg/gateways/{gateways[i]["gatewayId"]}/datasources/{gateways[i]["id"]}/status'
    
    response = requests.get(api_status, headers=headers)
    if response.ok:
        # response.json() is actually empty when it is a success
        
        status_base['datasourceId'] = gateways[i]["id"]
        status_base['gatewayId'] = gateways[i]["gatewayId"]
        status_base.get("error").get("pbi.error").get("details").append({"message":"success","detail":response.status_code})    
        status.append(status_base)
    else:
        results = json.loads(response.text)
        results["datasourceId"] = gateways[i]["id"]
        results["gatewayId"] = gateways[i]["gatewayId"]
        results.get("error").get("pbi.error").get("details").append({"message":"cannot connect","detail":response.status_code})    
        status.append(results)


In [ ]:
datasources = list()

for i in range(0,len(gateways)):
    api_datasource = f'https://api.powerbi.com/v1.0/myorg/gateways/{gateways[i]["gatewayId"]}/datasources/{gateways[i]["id"]}'
    response = requests.get(api_datasource, headers=headers)
    results = response.json()
    results.pop('@odata.context')
    datasources.append(results)


In [ ]:
import smtplib
from email.mime.text import MIMEText

sender_email = 'brcampb@microsoft.com'
receiver_email = 'brandonh.campbell@gmail.com'
subject = 'Hello from Python!'
body = 'This is a test email sent from a Python script.'

msg = MIMEText(body)
msg['Subject'] = subject
msg['From'] = sender_email
msg['To'] = receiver_email

# Set up the SMTP server (e.g., Gmail)
with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:
    server.login(sender_email, 'your_password')
    server.sendmail(sender_email, [receiver_email], msg.as_string())



sender_email = 'your_email@example.com'
receiver_email = 'recipient_email@example.com'
subject = 'Hello from Python!'
body = 'This is a test email sent from a Python script.'

msg = MIMEText(body)
msg['Subject'] = subject
msg['From'] = sender_email
msg['To'] = receiver_email

# Set up the SMTP server (e.g., Outlook)
with smtplib.SMTP('smtp.office365.com', 587) as server:
    server.starttls()
    server.login(sender_email, 'your_password')
    server.sendmail(sender_email, [receiver_email], msg.as_string())


In [ ]:
import smtplib
from email.mime.text import MIMEText

# Set up the SMTP server
smtp_server = 'smtp.gmail.com'
smtp_port = 587
sender_email = 'your_email@example.com'
password = 'your_password'

# Create a message
subject = 'Hello from Python!'
body = 'This is a test email sent from a Python script.'
msg = MIMEText(body)
msg['Subject'] = subject
msg['From'] = sender_email
msg['To'] = 'recipient_email@example.com'

# Send the email
with smtplib.SMTP(smtp_server, smtp_port) as server:
    server.starttls()
    server.login(sender_email, password)
    server.sendmail(sender_email, [msg['To']], msg.as_string())


In [11]:
# rest_api = "admin/capacities/refreshables"
# GET https://api.powerbi.com/v1.0/myorg/admin/groups?$expand=datasets
# htpps://api.powerbi.com/v1.0/myorg/admin/groups?$expand=datasets
rest_api = "https://api.powerbi.com/v1.0/myorg/admin/groups?$expand=datasets&$top=5000"

response = requests.get(url=rest_api, headers=headers)
result = response.json()


In [ ]:
result

In [12]:
# GET https://api.powerbi.com/v1.0/myorg/groups/{groupId}/datasets/{datasetId}/refreshes
for item in result['value']:
    group_id = item['id']
    for dataset in item['datasets']:
        dataset_id = dataset['id']
        if dataset['isRefreshable']==True:
            rest_api = f"https://api.powerbi.com/v1.0/myorg/groups/{group_id}/datasets/{dataset_id}/refreshes"

            response = requests.get(url=rest_api, headers=headers)
            if response.ok:
                results = response.json()
                for result in results['value']:
                    if len(result)>0:
                        print(result)



group_id 4a71fab9-5e5c-44d9-9340-096f4d19efff
group_id 78bf6589-a911-41be-becf-249d7164d1c5
group_id e9789ff4-0368-4877-b90a-7a2320dffc24
group_id 0352daf5-b0d6-4530-9d10-40e4990aaef7
group_id 1d688aa2-ef73-492f-8ea7-f7a0d8a9468a
group_id 1506ec5c-1b93-4ced-ba12-68079ede225e
group_id 23525a30-0123-4efa-a09d-b9a6e16c83f6
group_id 442cfda5-6975-42e9-93fe-743e7cc082a8
group_id aeada96a-aa68-4a77-b834-192a43dc098c
group_id 24c0bbb2-480d-480b-a75c-0eb59eb9dd9e
group_id ddaa6a53-9dbc-434b-8347-ba3feba99152
group_id 069c2b55-e75e-4f22-8ef1-33bbb866cb4d
group_id dd22b93f-aecb-4875-81ac-c0f340525840
group_id f49abe50-df50-4c83-9f2e-607b22bd9d10
group_id 18c0d3d1-97b9-4098-aba4-6f1c9f12b20c
group_id 49ae667e-be4d-4a76-9f2c-cced70751090
group_id c158b24e-275e-4e2f-9a67-404aa3533c5d
group_id d7e0162d-896d-450e-8bb8-9bc39aa9d88d
group_id 26be685f-f79f-4362-8910-d6f9a13f86fb
group_id d076fd46-0d50-4e7f-b93f-ed2d1fb405be
group_id 4e0157ef-273d-4f05-b44e-609c025cc603
group_id 7b472cbe-5bbd-4713-bbbf-c